<a href="https://colab.research.google.com/github/hagitli/repo/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive 
drive.mount('/mntDrive')
! ls "/mntDrive/My Drive"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive
'Colab Notebooks'  'Fruit Classification.gform'   trainData.csv
 DataSQLite.db	    testData.csv		 'טופס ללא כותרת.gform'


In [7]:

import sqlite3
from sqlite3 import Error
import csv

DB_FILE_PATH = '/mntDrive/My Drive/DataSQLite.db'
CSV_FILE_PATH = '/mntDrive/My Drive/testData.csv'

def connect_to_db(db_file):
    sqlite3_conn = None
    try:
        sqlite3_conn = sqlite3.connect(db_file)
        return sqlite3_conn
    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()

def insert_values_to_table(table_name, csv_file_path):

    conn = connect_to_db(DB_FILE_PATH)
    if conn is not None:
        c = conn.cursor()
        # Create table if it is not exist
        c.execute('CREATE TABLE IF NOT EXISTS ' + table_name +
                  '(review_ID VARCHAR,'
                  'user_id  VARCHAR,'
                  'business_id  VARCHAR,'
                  'ratings  INTEGER,'
                  'text VARCHAR)')

        # Read CSV file content
        values_to_insert = open_csv_file(csv_file_path)

        # Insert to table
        if len(values_to_insert) > 0:
            column_names, column_numbers = get_column_names_from_db_table(c, table_name)

            values_str = '?,' * column_numbers
            values_str = values_str[:-1]
            sql_query = 'INSERT INTO ' + table_name + '(' + column_names + ') VALUES (' + values_str + ')'
            c.executemany(sql_query, values_to_insert)
            conn.commit()

            print('SQL insert process finished')
        else:
            print('Nothing to insert')

        conn.close()

    else:
        print('Connection to database failed')


def open_csv_file(csv_file_path):
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)
        data = list()
        for row in reader:
            data.append(row)
        return data

def get_column_names_from_db_table(sql_cursor, table_name):

    table_column_names = 'PRAGMA table_info(' + table_name + ');'
    sql_cursor.execute(table_column_names)
    table_column_names = sql_cursor.fetchall()
    column_count = len(table_column_names)
    column_names = list()

    for name in table_column_names:
        column_names.append(name[1])

    return ', '.join(column_names), column_count

if __name__ == '__main__':
    insert_values_to_table('yelp', CSV_FILE_PATH)


SQL insert process finished


In [8]:
def create_small_table(conn):
  cur = conn.cursor()
  cur.execute('CREATE TABLE IF NOT EXISTS yelp_mod AS SELECT user_id,business_id,ratings FROM yelp')

def select_all(conn):
    cur = conn.cursor()
    cur.execute("SELECT count(*) FROM yelp_mod")
    rows = cur.fetchall() 
    for row in rows:
        print(row)

def main():
    # create a database connection
    conn = connect_to_db(DB_FILE_PATH)
    with conn:
        print("Query all")
        create_small_table(conn)
        select_all(conn)
 
if __name__ == '__main__':
    main()

Query all
(1052334,)


In [0]:
import pandas as pd
import numpy as np
# Create your connection.
conn = connect_to_db(DB_FILE_PATH)
#df = pd.read_sql_query("SELECT DISTINCT count(user_id) FROM yelp_mod", conn) #1052334
df_uid = pd.read_sql_query("SELECT DISTINCT user_id FROM yelp_mod", conn)
df_bid = pd.read_sql_query("SELECT DISTINCT business_id FROM yelp_mod", conn)
df_ratings = pd.read_sql_query("SELECT * FROM yelp_mod", conn)
df_ratings = df_ratings.reindex(columns = df_ratings.columns.tolist() + ['user_index','bus_index'])
#print(df_ratings.head())
#def create_matrix(df)

In [0]:

df_bid =np.transpose(df_bid)
#rating_matrix=df_bid@df_uid

In [17]:
print(df_bid.shape)
print(df_uid.shape)
#rating_matrix=np.dot(df_uid,df_bid)
#rating_matrix=np.outer(df_uid,df_bid)

(138544, 1)
(504159, 1)


In [16]:
import numpy as np
print(df_bid.head())
print(df_uid.head())
print(df_ratings.head(10))

              business_id
0  NCHS7mgfVAf6EbYVb2qSSQ
1  ZV1DPIxOWvYuU0lHPj7SDw
2  t10X85k_m8RuAZZgoMzvXg
3  NhNO2lN-p1RvA4MfR1wk1A
4  CVKOPzBVOj3_apFUmZ9ZWw
                  user_id
0  SQP77ENmGEyVD4uUmmvJNw
1  NtD-rHQYR0Ke2uMGArABoA
2  51SzvpuYXsIXdc13w0jcEQ
3  g_-8Wg_s7dw4CMzaY2CAMA
4  VDFXrJXhrNRBVUH68-yW4A
                  user_id             business_id  stars  user_index  bus_index
0  SQP77ENmGEyVD4uUmmvJNw  NCHS7mgfVAf6EbYVb2qSSQ      2         NaN        NaN
1  NtD-rHQYR0Ke2uMGArABoA  ZV1DPIxOWvYuU0lHPj7SDw      4         NaN        NaN
2  51SzvpuYXsIXdc13w0jcEQ  t10X85k_m8RuAZZgoMzvXg      3         NaN        NaN
3  g_-8Wg_s7dw4CMzaY2CAMA  NhNO2lN-p1RvA4MfR1wk1A      4         NaN        NaN
4  VDFXrJXhrNRBVUH68-yW4A  CVKOPzBVOj3_apFUmZ9ZWw      5         NaN        NaN
5  wYS8IObqNIFbpPwDkDTpFA  XTYEhwaaO_YjXcOGL_uBIQ      5         NaN        NaN
6  4xr9F4DK5anCTDZzLjfkiQ  CGd8erq6Ojn834pQ0gpRqA      5         NaN        NaN
7  9SXy2gvJkTAB9_tLQM-4bQ  u_s0zWoTlX7PoYDjtKAK2

In [30]:
print(df_ratings['user_id'][1])
print(df_uid['user_id'][1])

NtD-rHQYR0Ke2uMGArABoA
NtD-rHQYR0Ke2uMGArABoA


In [38]:
for i in range(len(df_ratings)):
  for j in range(len(df_uid)):
    if df_ratings['user_id'][i]==df_uid['user_id'][j]:
      df_ratings['user_index'][i]=j


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


KeyboardInterrupt: ignored

In [49]:
#RMSE Calculation
import numpy as np
import math
def rmse(pred,target):
  result=0
  temp_sum=0
  for i in range(len(pred)-1):
    temp_sum+=((pred[i]-target[i])**2)*target[i]
  result=temp_sum/len(pred)
  return math.sqrt(result)

#a=np.array([2, 5, 1])
#b=np.array([2, 1, 4])
#print (rmse(a,b))

3
2.309401076758503


In [1]:
print (rating_matrix.shape)

NameError: ignored

In [44]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse2 = sqrt(mean_squared_error(a, b))

print(rmse2)

2.886751345948129
